Isak Schwartz: 6 hours
Anton Forsberg: 6 hours
# Assignment 4
## notes
naive bayes
train given spam, ham


In [1]:
#Import scikit-learn dataset library
from sklearn import datasets

# Import train_test_split function
from sklearn.model_selection import train_test_split

#Import Gaussian Naive Bayes model
from sklearn.naive_bayes import GaussianNB

#Import scikit-learn metrics module for accuracy calculation
from sklearn import metrics

import numpy as np

import pandas as pd

import os

import re

from sklearn.feature_extraction.text import CountVectorizer

from sklearn.naive_bayes import MultinomialNB

from sklearn.naive_bayes import BernoulliNB

## 1: Preprocessing

In [2]:
def createTokens(pathsAndTargets, filterBool): # crates a list of tokens for all files in a given directory   
    files = []
    target = []
    for pathAndTarget in pathsAndTargets:
        
        for file in os.listdir(pathAndTarget[0]):
            filePath = (pathAndTarget[0] + "/" ) + file
            try:
                unFilteredFile = open(filePath,'r', encoding="utf-8").read()
                filteredFile = filetrFile(unFilteredFile)
                files.append(filteredFile)
                target.append(pathAndTarget[1])
            except UnicodeDecodeError:
                pass
            
    vectorizer = CountVectorizer(analyzer='word')
    # make vector from documents
    dataset = vectorizer.fit_transform(files).toarray()
    if(filterBool):
        filterList = filterWords(vectorizer.get_feature_names(), dataset)
       # print(filterList)
        vectorizer = CountVectorizer(analyzer='word', stop_words = frozenset(filterList))
        vectorizer._validate_vocabulary()
        dataset = vectorizer.fit_transform(files).toarray()
        # print(vectorizer.get_feature_names())
    
    return (dataset, target)

In [3]:
def printBayesianAccuracies(pathsAndTargets, filterBool):
    dataset = createTokens(pathsAndTargets, filterBool)
    ham = []
    hamTarget = []
    spam = []
    spamTarget = []
    
    for index in range(0, len(dataset[0])):
        if(0 == (dataset[1][index])):
            spam.append(dataset[0][index])
            spamTarget.append(0)
        else:
            ham.append(dataset[0][index])
            hamTarget.append(1)
            
            
    # Split dataset into training set and test set (70-30)
    #X_train, X_test, y_train, y_test = train_test_split(dataset[0], dataset[1], test_size=0.3,random_state=0)
    ham_train, ham_test, hamTarget_train, hamTarget_test = train_test_split(ham, hamTarget, test_size=0.3,random_state=0)
    spam_train, spam_test, spamTarget_train, spamTarget_test = train_test_split(spam, spamTarget, test_size=0.3,random_state=0)

    X_train = ham_train + spam_train
    y_train = hamTarget_train + spamTarget_train
    
    X_test = ham_test + spam_test
    y_test = hamTarget_test + spamTarget_test
    
    #Create a Naïve Bayes Classifiers
    classifiers = {(MultinomialNB(),"Multinomial naive bayes"), (BernoulliNB(binarize = 0), "Bernoulli naive bayes")}
    
    for classifier in classifiers:
        dataset = classifier[0]
        
        #Train the model using the training sets
        dataset.fit(X_train, y_train)

        #Predict the response for test dataset
        y_pred = dataset.predict(X_test)
        
        #Predict the response for ham dataset
        y_pred_ham = dataset.predict(ham_test)
        
        #Predict the response for spam dataset
        y_pred_spam = dataset.predict(spam_test)

        # Model Accuracy, how often is the classifier correct?
        print("Accuracy of " + classifier[1] + ": ",metrics.accuracy_score(y_test, y_pred))
        
        # Model Accuracy, how often is the classifier correct?
        #print("Accuracy of" + classifier[1] + " on ham : ",metrics.accuracy_score(hamTarget_test, y_pred_ham))
        
        # Model Accuracy, how often is the classifier correct?
        #print("Accuracy of" + classifier[1] + " on spam: ",metrics.accuracy_score(spamTarget_test, y_pred_spam))
        
        

In [4]:
def wordTable(words, occurences):
    # give length equal to the data entries
    totalOccurences = np.zeros(len(occurences[0]))
    for mail in occurences:
        for i in range (len(mail)):
            totalOccurences[i] += mail[i]
    # make table
    d = {'word': words, 'occurences': totalOccurences}
    df = pd.DataFrame(data=d)
    df.sort_values(by=['occurences'], inplace = True)
    print(df)

In [5]:
def filterWords(words, occurences):
    # give length equal to the data entries
    totalOccurences = np.zeros(len(occurences[0]))
    for mail in occurences:
        for i in range (len(mail)):
            totalOccurences[i] += mail[i]
    # make filterList
    d = {'word': words, 'occurences': totalOccurences}
    df = pd.DataFrame(data=d)
    unfrequent = df.loc[df['occurences'] < 4]['word'].values # filter away words occuring less than 10 times
    frequent = df.loc[df['occurences'] > 1000]['word'].values # filter away words occuring more than 10 times
    #print(type(unfrequent))
    return np.concatenate((unfrequent , frequent))

In [6]:
def filetrFile(file):
    try: 
        index = file.index("<!-- ### footer ### -->") # remove the footer
        file = file[:index]
    except ValueError:
        pass
    try: 
        index = file.index("<!-- ### \header ### -->")+ len("<!-- ### \header ### -->") # remove header
        file = file[index:]
    except ValueError:
        pass

    try:
        index = file.index("\n\n") # remove header for non html emails
        file = file[index:]
    except ValueError:
        pass

    try:
        index = file.index("<html>")+ len("<html>") # remove header
        file = file[index:]
    except ValueError:
        pass
    
    return file


In [7]:
# preprocessor function to remove html tags
def cleanhtml(raw_html):
    cleanr = re.compile('<.*?>')
    cleantext = re.sub(cleanr, ' ', raw_html)
    return cleantext

### 1.a : "run" entire text, including header and footer

In [ ]:
easyPaths = {("spam_2",0),("easy_ham_2",1)}
hardPaths = {("spam_2",0),("hard_ham",1)}
    
print("spam_2 and easy_ham_2 Accuracies:")
printBayesianAccuracies(easyPaths, True)
print("spam_2 and hard_ham Accuracies:")
printBayesianAccuracies(hardPaths, True)

#paths = {("spam_2",0),("easy_ham_2",1)}

#dataset = createTokens(paths, True) 

spam_2 and easy_ham_2 Accuracies:
Accuracy of Bernoulli naive bayes:  0.951885565669701
Accuracy of Multinomial naive bayes:  0.9778933680104032
spam_2 and hard_ham Accuracies:


### 1.b : split spam and ham in training / test sets

## 2 : python program

### 2.a : use 4 sets hamtrain, spamtrain, hamtest, spamtest

### 2.b : use naive bayes, classify test sets, report % ham, spam

## 5